In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import solve_for_masses as em
%matplotlib inline

As an example load data from the CKS-survey csv file which is available here:
https://california-planet-search.github.io/cks-website/ and then load it into the data structure required to calculate the masses.

In [2]:
data = pd.read_csv('cks_physical_merged.csv')
data.head()

,Unnamed: 0,id_starname,id_kic,id_koicand,id_kepler_name,koi_disposition,koi_period,koi_period_err1,koi_period_err2,koi_time0,...,iso_prad_err2,iso_sma,iso_sma_err1,iso_sma_err2,iso_insol,iso_insol_err1,iso_insol_err2,iso_teq,iso_teq_err1,iso_teq_err2
0,0,K00001,11446443,K00001.01,Kepler-1 b,CONFIRMED,2.470613,1.900000e-08,-1.900000e-08,2454955.763,...,-1.419266,0.035951,0.000596,-0.000596,890.712853,184.876216,-184.876216,1392.188224,71.758330,-71.758330
1,1,K00002,10666592,K00002.01,Kepler-2 b,CONFIRMED,2.204735,3.800000e-08,-3.800000e-08,2454954.359,...,-2.018515,0.036880,0.000727,-0.000727,3029.593093,931.186264,-931.186264,1890.643307,146.140168,-146.140168
2,2,K00003,10748390,K00003.01,Kepler-3 b,CONFIRMED,4.887803,1.770000e-07,-1.770000e-07,2454957.813,...,-0.410257,0.052952,0.000883,-0.000883,116.907786,20.094411,-20.094411,837.962116,35.981752,-35.981752
3,3,K00006,3248033,K00006.01,NaN,FALSE POSITIVE,1.334104,7.070000e-07,-7.070000e-07,2454966.702,...,-21.966014,0.025383,0.000427,-0.000427,3595.445148,694.155894,-694.155894,1973.338972,95.179897,-95.179897
4,4,K00007,11853905,K00007.01,Kepler-4 b,CONFIRMED,3.213669,1.122000e-06,-1.122000e-06,2454956.612,...,-0.596032,0.044247,0.001075,-0.001075,1233.844672,367.335736,-367.335736,1510.353647,112.889880,-112.889880


In [3]:
# empty list with all the systems

planet_systems = []

system_counter = 0
new_system = True

for index, row in data.iterrows():
    if (row['koi_disposition'] != 'FALSE POSITIVE'):
        if (index == 0):
            planet_systems.append(em.psystem(row['id_kic']))
        else:
            if (planet_systems[system_counter].name != row['id_kic']):
#                planet_systems[system_counter].update_planet_info()
#                planet_systems[system_counter].above_or_below_valley()
                if (planet_systems[system_counter].number_of_planets == 1):
                    # remove planet from list and don't update system_counter
                    del planet_systems[system_counter]
                else:
                    system_counter +=1
                new_system = True   
                planet_systems.append(em.psystem(row['id_kic']))
          
        if (new_system):
            planet_systems[system_counter].star.mass = row['iso_smass']
            planet_systems[system_counter].star.radius = row['iso_srad']
            planet_systems[system_counter].star.Teff = row['iso_steff']
            planet_systems[system_counter].star.age = row['iso_sage'] * 1e9 / 1e6 # want age in Myr
            new_system = False
    
        # add planet
        planet_systems[system_counter].add_planet(row['id_koicand'],row['iso_prad'],row['koi_period'])

Now compute which systems have rocky planets and which have not, compute the masses for the rocky planets and remove any multi-planet systems that only have gaseous or rocky planets. 

In [4]:
Xiron = 1./3.
Xice = 0.
Tmdot_Myr = 100.

mixed_systems = em.setup_systems(planet_systems,Tmdot_Myr,Xiron,Xice)

Now run through the mixed_systems and calculate the minimum masses, or indeed if they do not have one.

In [ ]:
em.estimate_min_masses(mixed_systems,Tmdot_Myr,Xiron,Xice)

/Users/jowen/anaconda/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/jowen/anaconda/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/jowen/anaconda/lib/python2.7/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


In [6]:
counter = 0
total_planets=0
for system in mixed_systems:
    print("System is", system.planets[0].name)
    for planet in system.planets:
        if planet.rocky_or_gaseous == 0:
            print(planet.min_mass_converged)
            
            if (planet.min_mass_converged == -2):
                    counter +=1
                    
            
            total_planets +=1
            print(planet.min_mass)
            print(planet.radius)
            
print('Number of inconsistent planets')
print(counter)
print('Fraction is')
print(counter*1./(total_planets*1.))
    

('System is', 'K00041.01')
-2
[-19.44312689]
2.3743738113
('System is', 'K00046.01')
1
8.80330855335
5.69830050599
('System is', 'K00070.01')
1
3.40197069245
3.10098924285
-2
[-12.4529254]
2.15716558913
1
0.41049554031
2.68703216927
('System is', 'K00072.01')
-5
-0.1
2.29553351659
('System is', 'K00082.01')
1
2.51171659214
2.56077601298
('System is', 'K00085.01')
-2
[-27.93107502]
2.55966891148
('System is', 'K00094.04')
1
2.79486096044
3.85105210707
1
0.940688860251
6.11575990228
1
3.92673302839
10.2998756201
('System is', 'K00102.01')
1
4.0996223315
3.57989367793
('System is', 'K00112.01')
1
0.210932220654
2.96014116511
('System is', 'K00115.01')
1
6.90066553269
2.6132098486
('System is', 'K00116.02')
1
0.844042662232
2.69028057864
1
2.56476992649
2.4505962132
('System is', 'K00117.01')
1
2.03016468312
3.48873936984
-2
[-10.79720612]
2.09010525763
('System is', 'K00156.01')
1
5.61774283705
2.75775081726
('System is', 'K00159.01')
1
0.263933895523
2.42158128263
('System is', 'K00191.0